# The Cheapest Airline Connection

COMPANY X employees are trying to find the cheapest flights to upcoming conferences. When people fly long distances, a direct city-to-city flight is often more expensive than taking two flights with a stop in a hub city. Travelers might save even more money by breaking the trip into three flights with two stops. But for the purposes of this challenge, let's assume that no one is willing to stop three times! You have a table with individual airport-to-airport flights, which contains the following columns:


• id - the unique ID of the flight;
• origin - the origin city of the current flight;
• destination - the destination city of the current flight;
• cost - the cost of current flight.


Your task is to produce a trips table that lists all the cheapest possible trips that can be done in two or fewer stops. This table should have the columns origin, destination and total_cost (cheapest one). Sort the output table by origin, then by destination. The cities are all represented by an abbreviation composed of three uppercase English letters. Note: A flight from SFO to JFK is considered to be different than a flight from JFK to SFO.


Example of the output:
origin | destination | total_cost
DFW | JFK | 200

In [93]:
import pandas as pd
import numpy as np
q=pd.read_csv('da_flights.csv')

,id,origin,destination,cost
0,1,SFO,JFK,500
1,2,SFO,DFW,200
2,3,SFO,MCO,400
3,4,DFW,MCO,100
4,5,DFW,JFK,200
5,6,JFK,LHR,1000


In [94]:
# merge 2 tables to create 1_stop_flight (the destination of fist flight will be the origin of the second flight)
q1=pd.merge(q,q,left_on='destination',right_on='origin',how='left')
q1

,id_x,origin_x,destination_x,cost_x,id_y,origin_y,destination_y,cost_y
0,1,SFO,JFK,500,6.0,JFK,LHR,1000.0
1,2,SFO,DFW,200,4.0,DFW,MCO,100.0
2,2,SFO,DFW,200,5.0,DFW,JFK,200.0
3,3,SFO,MCO,400,NaN,NaN,NaN,NaN
4,4,DFW,MCO,100,NaN,NaN,NaN,NaN
5,5,DFW,JFK,200,6.0,JFK,LHR,1000.0
6,6,JFK,LHR,1000,NaN,NaN,NaN,NaN


In [77]:
# merge 2 tables to create 2_stop_flight (the destination of second flight will be the origin of the third flight)
q2=pd.merge(q1,q,left_on='destination_y',right_on='origin',how='left')

In [79]:
# fill the na with the zero to make operation function
q2.cost_y=q2.cost_y.fillna(0)
q2.cost=q2.cost.fillna(0)

,id_x,origin_x,destination_x,cost_x,id_y,origin_y,destination_y,cost_y,id,origin,destination,cost
0,1,SFO,JFK,500,6.0,JFK,LHR,1000.0,NaN,NaN,NaN,NaN
1,2,SFO,DFW,200,4.0,DFW,MCO,100.0,NaN,NaN,NaN,NaN
2,2,SFO,DFW,200,5.0,DFW,JFK,200.0,6.0,JFK,LHR,1000.0
3,3,SFO,MCO,400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,DFW,MCO,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,DFW,JFK,200,6.0,JFK,LHR,1000.0,NaN,NaN,NaN,NaN
6,6,JFK,LHR,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
# Create the total cost of all flight (include 1 stop flight, 2 stops flight )
q2['total_cost']=q2['cost_x']+q2['cost_y']+q2['cost']

In [84]:
# fill the missing value from other column
q2.destination_y=q2['destination_y'].fillna(q2['destination_x'])
q2.destination=q2.destination.fillna(q2['destination_y'])

In [114]:
# drop the redundant columns and rename other columns
q2.drop(columns=['cost_x','destination_x','id_y','origin_y','destination_y','cost_y','id','origin','cost']).rename(columns={'id_x':'id','origin_x':'origin'})
q3=q2.drop(columns=['cost_x','destination_x','id_y','origin_y','destination_y','cost_y','id','origin','cost']).rename(columns={'id_x':'id','origin_x':'origin','total_cost':'cost'})

,id,origin,destination,cost
0,1,SFO,LHR,1500.0
1,2,SFO,MCO,300.0
2,2,SFO,LHR,1400.0
3,3,SFO,MCO,400.0
4,4,DFW,MCO,100.0
5,5,DFW,LHR,1200.0
6,6,JFK,LHR,1000.0


In [119]:
# combine to find all the all flight tabbles
q4=pd.concat([q3,q]).drop(columns=['id'])
q4

,origin,destination,cost
0,SFO,LHR,1500.0
1,SFO,MCO,300.0
2,SFO,LHR,1400.0
3,SFO,MCO,400.0
4,DFW,MCO,100.0
5,DFW,LHR,1200.0
6,JFK,LHR,1000.0
0,SFO,JFK,500.0
1,SFO,DFW,200.0
2,SFO,MCO,400.0


In [121]:
# find the min of flight as the final result
q4.groupby(['origin','destination'])['cost'].min().reset_index()

,origin,destination,cost
0,DFW,JFK,200.0
1,DFW,LHR,1200.0
2,DFW,MCO,100.0
3,JFK,LHR,1000.0
4,SFO,DFW,200.0
5,SFO,JFK,500.0
6,SFO,LHR,1400.0
7,SFO,MCO,300.0
